In [1]:
import pandas as pd
import numpy as np
from transformers import TFBertForSequenceClassification, BertTokenizer

D:\Anaconda\envs\stock_sentiment_predictor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 뉴스 감정 분석 모델 적용
- `뉴스 감정 데이터 셋`과 `KLUE BERT`모델을 사용하여 학습시킨 `sentiment_analysis_model` 모델 적용

In [2]:
# 뉴스 감정 분석 모델과 토크나이저 불러오기
MODEL_PATH = "../subject_focus_finetuned_model"  # 모델이 저장된 경로
MODEL_NAME = "klue/bert-base"  # 모델 이름

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
model = TFBertForSequenceClassification.from_pretrained(MODEL_PATH)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ../subject_focus_finetuned_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


### 네이버 뉴스 크롤링 데이터 전처리
- 네이버 뉴스 API사용하여 `SK하이닉스` 키워드로 검색된 뉴스 데이터
- 한 번에 요청할 수 있는 데이터가 1000개 이므로 `AWS 리눅스 서버`를 통한 `crontab 스케줄러`로 1시간 마다 API요청을 스케줄링하여 데이터 수집
- 데이터 건수: 8681건
- 데이터 기간: 2024_11_04_09:00 ~ 2024_11_27_18:00
- 컬럼: `pub_date(%Y_%m_%d_%H:%S)`, `title`, `description`
- 뉴스 감정 특성상 뉴스 내용이 중복되거나 비슷한 뉴스가 존재하나,
이 또한 해당 뉴스 내용에 대한 이슈도라고 생각하여 중복 제거를 하지 않음.

In [3]:
# 네이버 뉴스 크롤링 데이터 불러오기
news_data = pd.read_csv('../data/naver_news_crawling_data.csv', header=None, names=['pub_date', 'title', 'description'])

- 결측값 제거: 수집과정에서 `description`컬럼의 값이 누락 된 2개의 데이터 제거
- `title`컬럼 제거
- `pub_data`, `description` 컬럼의 형식을 변환
- 데이터를 BERT 모델에 적합한 형태로 변환

In [4]:
# 데이터 셋 요약 확인
print('데이터 요약')
news_data.info()

# 결측값 확인
print('\n결측값 여부 :',news_data.isnull().values.any(), '\n')

# 'description' 컬럼에서 결측치 제거
news_data = news_data.dropna(subset=['description'])

# 'title' 컬럼(뉴스 제목) 제거
news_data = news_data.drop(columns=['title'])

# 'pub_date' 컬럼을 datetime 형식으로 변환
news_data['pub_date'] = pd.to_datetime(news_data['pub_date'], format='%Y_%m_%d_%H:%S', errors='coerce')

# 'description' 컬럼을 문자열 형식으로 변환
news_data['description'] = news_data['description'].astype(str)

# 데이터 셋 요약 확인
print('데이터 요약')
news_data.info()

print('\n총 데이터 수 :',len(news_data))

# 데이터 전처리 (BERT 모델 입력에 맞게 텍스트를 인코딩)
def encode_data(data):
    return tokenizer(
        data['description'].tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )

# 데이터 전처리
encodings = encode_data(news_data)

데이터 요약
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8682 entries, 0 to 8681
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pub_date     8682 non-null   object
 1   title        8682 non-null   object
 2   description  8680 non-null   object
dtypes: object(3)
memory usage: 203.6+ KB

결측값 여부 : True 

데이터 요약
<class 'pandas.core.frame.DataFrame'>
Index: 8680 entries, 0 to 8681
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   pub_date     8680 non-null   datetime64[ns]
 1   description  8680 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 203.4+ KB

총 데이터 수 : 8680


### 감정 레이블 예측
- `description`컬럼을 대상으로 감정 레이블을 예측 (0:neutral ,1:positive ,2:negative)
- 예측한 감정 레이블을 데이터프레임에 추가하고 새로운 데이터를 CSV파일로 저장

In [5]:
# 예측 실행
predictions = model.predict(encodings['input_ids'])
predicted_classes = np.argmax(predictions.logits, axis=1)

272/272 [==============================] - 59s 194ms/step


In [7]:
# 예측된 감정 레이블을 데이터프레임에 추가
news_data['sentiment'] = predicted_classes

for label in range(3):
    if label == 0:
        sentiment = '중립'
    elif label == 1:
        sentiment = '긍정'
    else:
        sentiment = '부정'
    
    print(f"'{sentiment}'감정 데이터")
    print(news_data[news_data['sentiment'] == label].tail(3))
    print("\n")

# 예측된 결과를 새로운 CSV 파일로 저장
news_data.to_csv('../data/naver_news_crawling_data_with_sentiment.csv', index=False)

'중립'감정 데이터
                pub_date                                        description  \
8679 2024-11-27 18:00:00  이투데이박민웅 기자  연간 고정배당 25 상향 설비투자 원칙 등 밸류업 SK하이닉스...   
8680 2024-11-27 18:00:00  SK하이닉스가 내년부터 2027년까지 적용할 새로운 주주환원 정책과 기업가치 제고밸...   
8681 2024-11-27 18:00:00  SK하이닉스가 주당 연간 고정배당금을 기존 1200원에서 1500원으로 상향한다 순...   

      sentiment  
8679          0  
8680          0  
8681          0  


'긍정'감정 데이터
                pub_date                                        description  \
8638 2024-11-27 18:00:00  기존 1200원에서 1500원으로 상향 조정 SK하이닉스가 내년부터 3년간 적용할 ...   
8639 2024-11-27 18:00:00  SK하이닉스는 27일 주당 고정배당금을 상향하는 밸류업기업가치 제고 계획을 공시했다...   
8675 2024-11-27 18:00:00  SK하이닉스가 2027년까지 3년간 주당 연간 고정배당금을 기존 1200원에서 15...   

      sentiment  
8638          1  
8639          1  
8675          1  


'부정'감정 데이터
                pub_date                                        description  \
8619 2024-11-27 18:00:00  SK하이닉스솔리다임 포함는 2분기보다 15p 떨어진 206의 점유율로 2위를 차지했...   
8629 2024-11-27 18:00:00  S